In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import re
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
exit_notes = pd.read_excel('../data/TEIS Exit Data for NSS.xlsx', 3)

In [5]:
exit_notes

,POE,Child ID,DOB,Notification Date,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date,Days from I-IFSP to Exit,Exit Note
0,UC,471738,2019-02-28,2019-10-17,2019-10-29,2019-10-31,2020-05-12,618 - Parent withdraw,2020-09-21,326,* No Exit ECO available due to COVID-19
1,FT,449030,2018-07-22,2018-11-07,2018-12-03,2018-12-18,2021-03-04,618 - Moved out of state,2021-03-06,809,"***family moving out of state March 2021, and ..."
2,FT,463311,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,**Completed the transition planning conference...
3,FT,463313,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,**Completed the transition planning conference...
4,FT,469194,2017-08-31,2019-09-04,2019-10-02,2019-10-11,2020-05-26,618 - Part B eligibility not determined,2020-08-30,324,**Due to the impact of the COVID-19 pandemic a...
...,...,...,...,...,...,...,...,...,...,...,...
5577,MD,491609,2018-12-22,2020-10-20,2020-10-30,2020-11-09,2021-03-05,618 - Attempts to contact unsuccessful,2021-05-17,189,NaN
5578,GN,491743,2018-06-05,2020-10-22,2020-11-05,2020-11-09,2021-05-04,618 - Part B eligibility not determined,2021-06-04,207,NaN
5579,ET,491879,2019-04-24,2020-10-26,2020-11-02,2020-11-04,2021-04-15,618 - Parent withdraw,2021-05-06,183,NaN
5580,SC,492267,2018-06-01,2020-10-30,2020-11-17,2020-11-24,2021-04-15,618 - Part B eligibility not determined,2021-05-31,188,NaN


In [6]:
exit_notes.describe()

,Child ID,Days from I-IFSP to Exit
count,5582.000000,5582.000000
mean,460168.405231,504.307417
std,17611.033346,245.392985
min,420959.000000,180.000000
25%,446308.500000,308.000000
50%,464528.000000,454.000000
75%,473850.750000,670.000000
max,492408.000000,1076.000000


In [21]:
exit_notes['exit_dates'] = exit_notes['Exit Note'].str.findall(r'Exit Date: \d+[/-]\d+[/-]\d+')
exit_notes['reopened'] = exit_notes['Exit Note'].str.findall(r'Reopened: \d+[/-]\d+[/-]\d+')
exit_notes.head(30)

,POE,Child ID,DOB,Notification Date,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date,Days from I-IFSP to Exit,Exit Note,extra_dates,exit_dates,reopened
0,UC,471738,2019-02-28,2019-10-17,2019-10-29,2019-10-31,2020-05-12,618 - Parent withdraw,2020-09-21,326,* No Exit ECO available due to COVID-19,[],[],[]
1,FT,449030,2018-07-22,2018-11-07,2018-12-03,2018-12-18,2021-03-04,618 - Moved out of state,2021-03-06,809,"***family moving out of state March 2021, and ...",[],[],[]
2,FT,463311,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,**Completed the transition planning conference...,[],[],[]
3,FT,463313,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,**Completed the transition planning conference...,[],[],[]
4,FT,469194,2017-08-31,2019-09-04,2019-10-02,2019-10-11,2020-05-26,618 - Part B eligibility not determined,2020-08-30,324,**Due to the impact of the COVID-19 pandemic a...,[],[],[]
5,UC,473474,2017-11-05,2020-04-06,2020-04-24,2020-05-06,2020-06-23,618 - Part B eligibility not determined,2020-11-04,182,**LEA county on the Transition page and demogr...,"[04/06/2020, 11/26/2019]",[Exit Date: 11/26/2019],[Reopened: 04/06/2020]
6,NW,460887,2017-07-19,2019-04-17,2019-05-22,2019-05-29,2020-05-20,618 - Not eligible for Part B – Exit with no r...,2020-07-18,416,*Continuation of Transition Meeting Conference...,[11-5-19],[],[]
7,SE,480267,2018-01-30,2020-03-20,2020-04-28,2020-04-30,2020-10-06,618 - Parent withdraw,2021-01-19,264,01/19/2021- File closed. Parent withdraw due t...,[01/19/2021],[],[]
8,GN,469243,2019-07-26,2019-09-04,2019-10-01,2019-10-02,2020-03-19,618 - Parent withdraw,2021-02-11,498,02-11-2021 – PWN signed and returned for volun...,"[02-11-2021, 02/05/2021, 04/07/2020]",[Exit Date: 04/07/2020],[Reopened: 02/05/2021]
9,FT,438070,2018-05-22,2018-06-06,2018-07-03,2018-07-24,2021-03-19,618 - Not eligible for Part B – Exit with refe...,2021-05-21,1032,03/19/2021: The Transition Planning Conference...,[03/19/2021],[],[]
